In [3]:
import pandas as pd

df = pd.read_csv('steam-200k.csv', header=None)
df.columns = ['user_id', 'game_title', 'behavior_name', 'value', 'unused']
df = df.drop('unused', axis=1)
play_df = df[df['behavior_name'] == 'play'].copy()

play_df = play_df.drop('behavior_name', axis=1)

play_df.rename(columns={'user_id': 'user', 'game_title': 'item', 'value': 'rating'}, inplace=True)


import numpy as np
play_df['rating'] = np.log1p(play_df['rating'])

print("5 dòng đầu của dữ liệu đã xử lý:")
print(play_df.head())

5 dòng đầu của dữ liệu đã xử lý:
        user                        item    rating
1  151603712  The Elder Scrolls V Skyrim  5.613128
3  151603712                   Fallout 4  4.477337
5  151603712                       Spore  2.766319
7  151603712           Fallout New Vegas  2.572612
9  151603712               Left 4 Dead 2  2.292535


In [5]:
rating_sparse_df = play_df.pivot_table(
    index='user',
    columns='item',
    values='rating',
    aggfunc='sum'
).fillna(0)

print("\n5 dòng đầu của ma trận thưa:")
print(rating_sparse_df.head())


5 dòng đầu của ma trận thưa:
item    007 Legends  0RBITALIS  \
user                             
5250            0.0        0.0   
76767           0.0        0.0   
86540           0.0        0.0   
144736          0.0        0.0   
181212          0.0        0.0   

item    1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)  \
user                                                                 
5250                                                  0.0            
76767                                                 0.0            
86540                                                 0.0            
144736                                                0.0            
181212                                                0.0            

item    10 Second Ninja  10,000,000  100% Orange Juice  1000 Amps  \
user                                                                
5250                0.0         0.0                0.0        0.0   
76767               0.0         0

In [6]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy


reader = Reader(rating_scale=(play_df['rating'].min(), play_df['rating'].max()))

data = Dataset.load_from_df(play_df[['user', 'item', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=.25)

sim_options = {
    'name': 'cosine',
    'user_based': True
}
algo = KNNBasic(sim_options=sim_options)

algo.fit(trainset)

predictions = algo.test(testset)

print("\nĐánh giá mô hình bằng Surprise:")
accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.

Đánh giá mô hình bằng Surprise:
RMSE: 1.4368


1.4368271770571601

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

train_df, test_df = sklearn_train_test_split(play_df, test_size=0.25, random_state=42)

train_user_item_matrix = train_df.pivot_table(index='user', columns='item', values='rating', aggfunc='sum').fillna(0)

user_similarity = cosine_similarity(train_user_item_matrix)
user_sim_df = pd.DataFrame(user_similarity, index=train_user_item_matrix.index, columns=train_user_item_matrix.index)

def predict_rating(user_id, item_id, k=25):
    if user_id not in train_user_item_matrix.index or item_id not in train_user_item_matrix.columns:
        return train_df['rating'].mean()
    user_sims = user_sim_df[user_id].drop(user_id)
    users_who_rated_item = train_user_item_matrix[train_user_item_matrix[item_id] > 0].index

    neighbor_sims = user_sims[user_sims.index.isin(users_who_rated_item)]

    top_k_neighbors = neighbor_sims.nlargest(k)

    if top_k_neighbors.empty:
        return train_df['rating'].mean()

    numerator = 0
    denominator = 0
    for neighbor_id, similarity in top_k_neighbors.items():
        neighbor_rating = train_user_item_matrix.loc[neighbor_id, item_id]
        numerator += similarity * neighbor_rating
        denominator += similarity

    if denominator == 0:
        return train_df['rating'].mean()

    return numerator / denominator

print("\nĐánh giá mô hình tự triển khai:")
test_df['predicted_rating'] = test_df.apply(lambda row: predict_rating(row['user'], row['item']), axis=1)

rmse = np.sqrt(mean_squared_error(test_df['rating'], test_df['predicted_rating']))
print(f"RMSE trên tập kiểm tra: {rmse}")


Đánh giá mô hình tự triển khai:
RMSE trên tập kiểm tra: 1.4567738813242601
